# Basic exploration of correlations between node- and word- vecs in SoSweet

In [1]:
import logging

# utilities and plotting
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb
# node2vec stuff
from gensim.models import Word2Vec

logging.basicConfig(level=logging.INFO)

In [15]:
network_emb = Word2Vec.load('data/sosweet-network/undir_weighted_mention_network_thresh_5.emb')
lang_emb = Word2Vec.load('data/sosweet-w2v/dim_50/lowe_dim_sosweet2vec.w2v')

INFO:gensim.utils:loading Word2Vec object from data/sosweet-network/undir_weighted_mention_network_thresh_5.emb
INFO:gensim.utils:loading wv recursively from data/sosweet-network/undir_weighted_mention_network_thresh_5.emb.wv.* with mmap=None
INFO:gensim.utils:setting ignored attribute syn0norm to None
INFO:gensim.utils:setting ignored attribute cum_table to None
INFO:gensim.utils:loaded data/sosweet-network/undir_weighted_mention_network_thresh_5.emb
INFO:gensim.utils:loading Word2Vec object from data/sosweet-w2v/dim_50/lowe_dim_sosweet2vec.w2v
INFO:gensim.utils:loading wv recursively from data/sosweet-w2v/dim_50/lowe_dim_sosweet2vec.w2v.wv.* with mmap=None
INFO:gensim.utils:loading syn0 from data/sosweet-w2v/dim_50/lowe_dim_sosweet2vec.w2v.wv.syn0.npy with mmap=None
INFO:gensim.utils:setting ignored attribute syn0norm to None
INFO:gensim.utils:loading syn1neg from data/sosweet-w2v/dim_50/lowe_dim_sosweet2vec.w2v.syn1neg.npy with mmap=None
INFO:gensim.utils:setting ignored attribute c

In [16]:
len(network_emb.wv.vocab)

178359

In [17]:
len(lang_emb.wv.vocab)

492519

TODODOC:
* users.txt: `cat data/sosweet-network/undir_weighted_mention_network_thresh_5.txt | scripts/pipe-unique_users > data/sosweet-network/undir_weigted_mention_network_thresh_5.users.txt`
* filtered tweets: `scripts/iter-gz "scripts/pipe-filter_users data/sosweet-network/undir_weighted_mention_network_thresh_5.users.txt" data/sosweet-text/undir_weighted_mention_network_thresh_5 data/sosweet-text/*.gz`

Jacobo's parsing to get the words out of the tweets (which we need to reproduce here):

```bash
for f in /datastore/complexnet/twitter/data/2017*.tgz
do
	echo $f
	zcat $f|sed "s/^[^{]*//g"| ./jq -r '.|[.actor.id,.body,.actor.followersCount,.actor.friendsCount]|@tsv'| sed '/RT /d;s/@[^ ]*/ /g;s/http[^ ]*/ /g'|sort -k1,1| uniq >ids_and_body.txt
	join -11 -21 $net_ids ids_and_body.txt -t $'\t'>> $net_txt
done

cat $net_txt| sort -k1,1 -b| tr "[:upper:]" "[:lower:]"|sed 's/[?!;,*+<>-]/ /g'|sed "s/'/ /g"|sed 's/"//g' > temp
sed -i "y/âāáǎàêēéěèîīíǐìôōóǒòûūúǔùǖǘǚǜÂĀÁǍÀÊĒÉĚÈÎĪÍǏÌÔŌÓǑÒÛŪÚǓÙǕǗǙǛ/aaaaaeeeeeiiiiiooooouuuuuuuuuaaaaaeeeeeiiiiiooooouuuuuuuuu/" temp
rm ids_and_body.txt
mv temp $net_txt
```

implement using https://stackoverflow.com/questions/517923/what-is-the-best-way-to-remove-accents-in-a-python-unicode-string#518232

In [39]:
import csv
import gzip
from collections import defaultdict
import re
import nltk

In [ ]:
url_regex = re.compile('http[^ ]')

def words(text):
    # Remove urls (which contain : and / characters)
    re.sub(url_regex, '', text)
    # TODO: transliterate accents
    # TODO: split on ?!;:,.<>'"`=+-*%$/\|()
    # TODO: remove stuff starting with @, and 'RT'

In [37]:
user_words = defaultdict(set)
with gzip.open('data/sosweet-text/undir_weighted_mention_network_thresh_5/'
               '2017-10-pipe-user_timestamp_body-csv-pipe-filter_users.gz', 'rt') as tweets_file:
    reader = csv.DictReader(tweets_file, fieldnames=['user_id', 'timestamp', 'body'])
    for i, tweet in enumerate(reader):
        user_words[int(tweet['user_id'])].update(words(tweet['body']))

KeyboardInterrupt: 

In [25]:
import unicodedata

In [33]:
lang_emb.wv[tweet['body'][-1]]

KeyError: "word '😂' not in vocabulary"

In [32]:
unicodedata.category('界')

'Lo'